In [1]:
#python
import os

#package
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import librosa
import IPython.display as ipd
import librosa.display
import scipy.io.wavfile

#keras
from keras.utils import np_utils

#sklearn
from sklearn.preprocessing import LabelEncoder

from scipy import signal
from tqdm import tqdm_notebook as tqdm

Using TensorFlow backend.


In [2]:
#Data directory
dir_list=os.listdir('E:/SER/audio/')
dir_list.sort()
print(dir_list)

['Actor_01', 'Actor_02', 'Actor_03', 'Actor_04', 'Actor_05', 'Actor_06', 'Actor_07', 'Actor_08', 'Actor_09', 'Actor_10', 'Actor_11', 'Actor_12', 'Actor_13', 'Actor_14', 'Actor_15', 'Actor_16', 'Actor_17', 'Actor_18', 'Actor_19', 'Actor_20', 'Actor_21', 'Actor_22', 'Actor_23', 'Actor_24']


In [3]:
#create Dataframe
ravdess_db=pd.DataFrame(columns=['path','source','actor','gender','emotion','emotion_lb'])
count=0
for i in dir_list:
    file_list=os.listdir('E:/SER/audio/'+i)
    for f in file_list:
        nm=f.split('.')[0].split('-')
        path='E:/SER/audio/'+i+'/'+f
        actor=int(nm[-1])
        emotion=int(nm[2])
        source="Ravdess"
            
        if int(actor)%2==0:
            gender="female"
        else:
            gender="male"
            
        if nm[3]=='01':
            intensity=0
        else:
            intensity=1
            
        if nm[4]=='01':
            statement=0
        else:
            statement=1
            
        if nm[5]=='01':
            repeat=0
        else:
            repeat=1
            
        if emotion==1:
            lb="neutral"
        elif emotion==2:
            lb="calm"
        elif emotion==3:
            lb="happy"
        elif emotion==4:
            lb="sad"
        elif emotion==5:
            lb="angry"
        elif emotion==6:
            lb="fearful"
        elif emotion==7:
            lb="disgust"
        elif emotion==8:
            lb="surprised"
        else:
            lb="none"
            
        ravdess_db.loc[count]=[path,source,actor,gender,emotion,lb]
        count+=1

In [4]:
ravdess_db.sort_values(by='path',inplace=True)
ravdess_db.index=range(len(ravdess_db.index))
ravdess_db.head()

,path,source,actor,gender,emotion,emotion_lb
0,E:/SER/audio/Actor_01/03-01-01-01-01-01-01.wav,Ravdess,1,male,1,neutral
1,E:/SER/audio/Actor_01/03-01-01-01-01-02-01.wav,Ravdess,1,male,1,neutral
2,E:/SER/audio/Actor_01/03-01-01-01-02-01-01.wav,Ravdess,1,male,1,neutral
3,E:/SER/audio/Actor_01/03-01-01-01-02-02-01.wav,Ravdess,1,male,1,neutral
4,E:/SER/audio/Actor_01/03-01-02-01-01-01-01.wav,Ravdess,1,male,2,calm


In [5]:
ravdess_db['split']=np.where((ravdess_db.actor==23) | (ravdess_db.actor==24), 'Test',
                              (np.where((ravdess_db.actor==21) | (ravdess_db.actor==22),'Val', 'Train')))

In [6]:
ravdess_db.loc[ravdess_db.emotion_lb=='calm',['emotion','emotion_lb']]=1,'neutral'

In [7]:
dataset_db=ravdess_db
dataset_db.emotion_lb=dataset_db.gender+"_"+dataset_db.emotion_lb

In [8]:
audio_duration=3
sampling_rate=44100
input_length=sampling_rate * audio_duration
n_mfcc = 20

In [9]:
data_sample = np.zeros(input_length)
MFCC = librosa.feature.mfcc(data_sample, sr=sampling_rate, n_mfcc=n_mfcc)

In [10]:
signal, sample_rate = librosa.load(dataset_db.path[0], res_type='kaiser_fast',sr=sampling_rate)
signal,index = librosa.effects.trim(signal,top_db = 25)
signal = scipy.signal.wiener(signal)

if len(signal) > input_length:
    signal = signal[0:input_length]
elif  input_length > len(signal):
    max_offset = input_length - len(signal)  
    signal = np.pad(signal, (0, max_offset), "constant")

In [11]:
signal = np.array(signal).reshape(-1,1)

In [12]:
audios= np.empty(shape=(dataset_db.shape[0],128, MFCC.shape[1], 1))

count=0
for i in tqdm(range(len(dataset_db))):
    signal, sample_rate = librosa.load(dataset_db.path[i], res_type='kaiser_fast',sr=sampling_rate)
    signal,index = librosa.effects.trim(signal,top_db = 25)
    signal = scipy.signal.wiener(signal)
    
    if len(signal) > input_length:
        signal = signal[0:input_length]
    elif  input_length > len(signal):
        max_offset = input_length - len(signal)  
        signal = np.pad(signal, (0, max_offset), "constant")

    melspec = librosa.feature.melspectrogram(signal, sr=sample_rate, n_mels=128,n_fft=2048,hop_length=512)   
    logspec = librosa.amplitude_to_db(melspec)
    logspec = np.expand_dims(logspec, axis=-1)
    audios[count,] = logspec 
    count+=1

C:\Users\ayush\AppData\Roaming\Python\Python37\site-packages\scipy\signal\signaltools.py:974: RuntimeWarning: divide by zero encountered in true_divide
  res *= (1 - noise / lVar)
C:\Users\ayush\AppData\Roaming\Python\Python37\site-packages\scipy\signal\signaltools.py:974: RuntimeWarning: invalid value encountered in multiply
  res *= (1 - noise / lVar)


In [13]:
x_train=audios[(dataset_db['split']=='Train')]
y_train=dataset_db.emotion_lb[(dataset_db['split']=='Train')]

print(x_train.shape,y_train.shape)

(1200, 128, 259, 1) (1200,)


In [14]:
x_test=audios[(dataset_db['split']=='Val')]
y_test=dataset_db.emotion_lb[(dataset_db['split']=='Val')]

print(x_test.shape,y_test.shape)

(120, 128, 259, 1) (120,)


In [15]:
x_train=np.array(x_train)
y_train=np.array(y_train)
x_test=np.array(x_test)
y_test=np.array(y_test)

In [16]:
lb=LabelEncoder()
y_train=np_utils.to_categorical(lb.fit_transform(y_train))
y_test=np_utils.to_categorical(lb.fit_transform(y_test))

In [17]:
x_traincnn=x_train
x_testcnn=x_test

In [18]:
x_traincnn.shape,x_testcnn.shape,y_train.shape,y_test.shape

((1200, 128, 259, 1), (120, 128, 259, 1), (1200, 14), (120, 14))

In [21]:
num_classes=len(np.unique(np.argmax(y_train,1)))
input_shape=x_traincnn.shape[1:]
print(input_shape)
learning_rate=0.0001
decay = 1e-6
momentum=0.9
print(num_classes)

(128, 259, 1)
14
